In [7]:

import yaml
import os
from glob import glob
import os
import pandas as pd
import json
import pickle
import datetime

import numpy as np
import json


## Reference train set, query test set

In [2]:
##FILTer OUT THE MODELS TO TRAIN

directory_of_models_trained_64 =  '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_**'
globbed_dir = glob(directory_of_models_trained_64)
print(globbed_dir)

replaced_models = ['non_permutated',"16_ids_monocolor_and",'16_ids_four_times_colors_batch','_4_ids_','double_colors','max']
wanted = []
for i in globbed_dir:
    found = False
    for j in replaced_models:
        
        if j in i:
            found =True
    if(not found):
        wanted.append(i)
        
directory_of_models_trained_all_ids = '/data/new_reid_models/summer_bee_dataset_open_train_bee_64_ids_batch1_*'

for i in glob(directory_of_models_trained_all_ids):
    wanted.append(i)
wanted

['/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_32', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_16', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_02', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_max', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_64', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_08', '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_04']


['/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_32',
 '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_16',
 '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_02',
 '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_64',
 '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_08',
 '/home/lmeyers/ReID_complete/color_detect_experiments/_ids_batch1_sample_num_04']

In [8]:
wanted = []
#wanted.append('/home/lmeyers/ReID_complete/color_detect_experiments/8_ids_double_colors_batch1_sample_num_64')
#wanted.append('/home/lmeyers/ReID_complete/color_detect_experiments/16_ids_four_times_permutated_colors_batch1_sample_num_64')
#wanted.append('/home/lmeyers/ReID_complete/color_detect_experiments/4_ids_batch1_sample_num_64')
wanted.append('/home/jrodriguez/color_detect_experiments/_ids_permutated_ids_batch1_sample_num_64')
wanted.append('/home/jrodriguez/color_detect_experiments/_ids_monocolor_and_permutated_ids_batch1_sample_num_64')


In [9]:

directory_for_ymls = './new_configs_reference_train_query_test/'
directory_for_jsons = './jsons_for_results/'

gpu_id = '0'
if(not os.path.exists(directory_for_ymls)):
    os.mkdir(directory_for_ymls)
    
if(not os.path.exists(directory_for_jsons)):
    os.mkdir(directory_for_jsons)


for model_dir in wanted:
    
    model_files = glob(model_dir+'/checkpoints/*00.pth')
    #model_files.append(glob(model_dir+'/checkpoints/*99.pth')[0])
    
    
    #should not happen if all models were trained
    if len(model_files) <=0:
        print('this path needs training')
        print(model_dir)
        continue
    
    yml_config_path = glob(model_dir+'/*.yml')[0]
    with open(yml_config_path, 'r') as f:
        yml_config = yaml.safe_load(f)
    
    for model in model_files:
        
        model_checkpoint = model.split('/')[-1][:-4]
        

        yml_config['model_settings']['model_path'] = model

        training_csv = yml_config['data_settings']['datafiles']['train']

        #training_csv = training_csv.replace('/home/gsantiago/summer_bee_data/','/home/gsantiago/all_data_csvs/')
        training_csv = training_csv.replace('/home/lizardi_lab/josue/summer_bee_data/', '/home/gsantiago/summer_bee_data/')

        ## this prints 

        #/home/gsantiago/summer_bee_data/open_sets/new_open_08_ids_monocolor_batch1

           # print(os.path.dirname(training_csv))
        parent_train_directory=os.path.dirname(training_csv)
        batch = parent_train_directory.split('_')[-1]

        # things needed for csv
        run_str  = os.path.basename(training_csv)[:-4]
        training_df = pd.read_csv(training_csv)
        num_ids = training_df['reID'].nunique()
        num_images_per_id = run_str.split('_')[-1]

        print(batch)
        print(run_str)
        print(num_ids)
        print(num_images_per_id)


        #reference  train, query test
        #if(batch == 'batch1'):
        open_sets_dir = (os.path.dirname(parent_train_directory))


        #reference_csv = yml_config['data_settings']['datafiles']['reference']

        #training_csv = training_csv.replace('/home/gsantiago/summer_bee_data/','/home/gsantiago/all_data_csvs/')
        #reference_csv = reference_csv.replace('/home/lizardi_lab/josue/summer_bee_data/', '/home/gsantiago/summer_bee_data/')
        
        test_csv = yml_config['data_settings']['datafiles']['test']
        test_csv = test_csv.replace('/home/lizardi_lab/josue/summer_bee_data/', '/home/gsantiago/summer_bee_data/')

        color_map_file = yml_config['data_settings']['datafiles']['color_map']
        color_map_file = color_map_file.replace('/home/lizardi_lab/josue/color_detect_experiments/', '/home/lmeyers/ReID_complete/')

        #reference_file_train_both = glob(open_sets_dir+'/open_reference_query_testing_batch1/*reference*')[0]
        #reference_file_test_both = glob(open_sets_dir+'/open_reference_query_testing_batch2/*reference*')[0]
        
        #query_file=glob(open_sets_dir+'/open_reference_query_testing_batch2/*query*')[0]

        """
        else:
            open_sets_dir = (os.path.dirname(parent_train_directory))
            reference_file_train_both = glob(open_sets_dir+'/open_reference_query_testing_batch2/*reference*')[0]
            reference_file_test_both = glob(open_sets_dir+'/open_reference_query_testing_batch1/*reference*')[0]
            
            query_file=glob(open_sets_dir+'/open_reference_query_testing_batch1/*query*')[0]
            #training_csv = training_csv.replace('/home/gsantiago/all_data_csvs/', '/home/gsantiago/summer_bee_data/')
        """

        #yml_config['data_settings']['datafiles']['reference']=reference_

        yml_config['data_settings']['datafiles']['test']=test_csv

        yml_config['eval_settings']['results_file']='/home/lmeyers/ReID_complete/auto_eval_models_color_detect/evaluation_all_checkpoints.csv'
        yml_config['eval_settings']['pickle_file'] = '/home/lmeyers/ReID_complete/auto_eval_models_color_detect/results_pickle.pkl'
        yml_config['train_settings']['gpu']=gpu_id

        yml_config['data_settings']['datafiles']['color_map'] = color_map_file

       

        yml_file = directory_for_ymls+ os.path.basename(yml_config_path)
        with open(yml_file, 'w') as f:
           yaml.dump(yml_config, f)


        ## python runs


        !python3 ../color_detect_experiments/color_detect_eval.py --config_file {yml_file}
        with open(yml_config['eval_settings']['pickle_file'],'rb') as fi:
            results = pickle.load(fi)  


        # Convert ndarray to list

        #for key in results.keys():
            #results[key] = results[key].tolist()

        with open(directory_for_jsons+run_str+'_training_'+model_checkpoint+'.json', 'w') as f:
            json.dump(results, f)

        # Write out run summary to results tracking document
        results_df = pd.read_csv(yml_config['eval_settings']['results_file'])
        results_df.loc[len(results_df)] = {'run_str': run_str,
                                           'type_of_reference': 'training',
                                            'num_ids':num_ids,
                                            'num_images_per_id':num_images_per_id,
                                            'total_training_images':len(training_df),
                                            'batch_size':yml_config['data_settings']['batch_size'],
                                            'num_epochs':model_checkpoint,
                                            'percent_correct_samples':results['percent_correct_samples'],
                                            'total_acc':results['Total Acc:'],
                                            'total_precision':results['Total Precision:'],
                                            'total_recall':results['Total Recall:'],
                                            'training_file':training_csv,
                                            'test_file':test_csv,
                                            'stop_epoch':results['stop_epoch'],
                                            }
                                            
        results_df.to_csv(yml_config['eval_settings']['results_file'],index=False)


    """  ##-----------Duplicate???
        
        yml_config['data_settings']['datafiles']['reference']=reference_file_test_both

        yml_config['data_settings']['datafiles']['query']=query_file
        yml_config['train_settings']['gpu']=gpu_id

        yml_file = directory_for_ymls+ os.path.basename(yml_config_path)
        with open(yml_file, 'w') as f:
            yaml.dump(yml_config, f)


        ## python runs


        !python3 ../pytorch_eval_reid.py --config_file {yml_file}
        with open(yml_config['eval_settings']['pickle_file'],'rb') as fi:
            results = pickle.load(fi)  


        # Convert ndarray to list

        for key in results.keys():
            results[key] = results[key].tolist()





        with open(directory_for_jsons+run_str+'testing'+model_checkpoint+'.json', 'w') as f:
            json.dump(results, f)

        # Write out run summary to results tracking document
        results_df = pd.read_csv(yml_config['eval_settings']['results_file'])
        results_df.loc[len(results_df)] = {'run_str': run_str,
                                            'type_of_reference': 'testing',
                                            'num_ids':num_ids,
                                            'num_images_per_id':num_images_per_id,
                                            'total_training_images':len(training_df),
                                            'batch_size':yml_config['data_settings']['batch_size'],
                                            'num_epochs':model_checkpoint,

                                            '1NN':results['1NN_acc'],
                                            '3NN':results['3NN_acc'],


                                            'training_file':training_csv,
                                            'reference_file':reference_file_test_both,
                                            'query_file':query_file
                                            }
        results_df.to_csv(yml_config['eval_settings']['results_file'],index=False)
    
    
    """    
    
    

batch1
summer_bee_dataset_open_train_bee_32_ids_permutated_ids_batch1_sample_num_64
32
64
2023-11-14 19:09:48.215402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 19:09:49.363523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Device: cuda
Evaluating model...
Traceback (most recent call last):
  File "/home/lmeyers/ReID_complete/auto_eval_models_color_detect/../color_detect_experiments/color_detect_eval.py", line 145, in <module>
    eval(args.config_file)
  File "/home/lmeyers/ReID_complete/auto_eval_models_color_detect/../color_detect_experiments/color_detect_eval.py", line 49, in eval
    test_dataset = ColorMap_w_Order(dft_test, 'filename', 'label',data_config['input_size'],'test',data_co

In [7]:
 with open(yml_config['eval_settings']['pickle_file'],'rb') as fi:
        results = pickle.load(fi)  
        
    
# Convert ndarray to list

for key in results.keys():
    print(key)


1NN_acc
3NN_acc
label_list
knn_class
knn_conf
